## H&M Recommender system - Capstone Part 8

“The theory of the “wisdom of crowds,” says that if you aggregate many different opinions from a diverse group of people, you are much more likely to arrive at the best opinion than if you just listen to one specialist.”
— Simon Kuper

The focus of this notebook is to investigate if ensembling can give rise to a better MAP@12 score as part of model selection since I have built several models by this juncture. It would be a waste to simply choose the best model without trying out this step.

Furthermore, most of the models built are rule based and do not have hyperparameters to tune, the scores of those models are more or less fixed and unlikely to further improve greatly. 

In [1]:
# Import libraries
import numpy as np
import pandas as pd
import gc

# To ensemble I used 5 submissions that gave me the highest scores so far

In [3]:
sub1 = pd.read_csv('../datasets/LGBM5w_NDCG@122025ES.csv').sort_values('customer_id').reset_index(drop=True)  # 0.0199
sub2 = pd.read_csv('../datasets/submission8.csv').sort_values('customer_id').reset_index(drop=True)           # 0.0206
sub3 = pd.read_csv('../datasets/trending.csv').sort_values('customer_id').reset_index(drop=True)              # 0.0226
sub4 = pd.read_csv('../datasets/age.csv').sort_values('customer_id').reset_index(drop=True)                   # 0.0227
sub5 = pd.read_csv('../datasets/kmeans.csv').sort_values('customer_id').reset_index(drop=True)                # 0.0224

In [6]:
sub1.columns = ['customer_id', 'prediction1']
sub1['prediction2'] = sub2['prediction']
sub1['prediction3'] = sub3['prediction']
sub1['prediction4'] = sub4['prediction']
sub1['prediction5'] = sub5['prediction']

del sub2, sub3, sub4, sub5
sub1.head()

,customer_id,prediction1,prediction2,prediction3,prediction4,prediction5
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,0568601043 0924243001 0909370001 0865799006 09...,0568601043 0751471001 0909370001 0918522001 09...,0568601043 0568601006 0448509014 0573085028 07...,0568601043 0568601006 0745232001 0751471001 04...,0568601043 0568601006 0745232001 0751471001 04...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,0924243001 0924243002 0918522001 0923758001 08...,0924243001 0924243002 0923758001 0918522001 09...,0826211002 0800436010 0739590027 0448509014 05...,0826211002 0739590027 0811835004 0764280001 07...,0826211002 0739590027 0811835004 0764280001 07...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0794321007 0924243001 0762846027 0918522001 09...,0794321007 0924243001 0923758001 0924243002 09...,0794321007 0852643001 0852643003 0858883002 07...,0794321007 0858883002 0852643003 0727808007 08...,0794321007 0852643001 0852643003 0727808007 08...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,0924243001 0924243002 0918522001 0923758001 08...,0924243001 0924243002 0923758001 0918522001 09...,0448509014 0573085028 0751471001 0706016001 06...,0751471001 0579541001 0573085028 0673677002 06...,0751471001 0678942001 0673677002 0579541001 05...
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,0924243001 0924243002 0918522001 0923758001 08...,0924243001 0924243002 0923758001 0918522001 09...,0730683050 0791587015 0896152002 0818320001 09...,0730683050 0791587015 0896152002 0927530004 08...,0730683050 0791587015 0896152002 0927530004 05...


In [7]:
def cust_blend(dt, W = [1,1,1,1,1]):
    #Global ensemble weights
    #W = [1.15,0.95,0.85]

    #Create a list of all model predictions
    REC = []

    REC.append(dt['prediction1'].split())
    REC.append(dt['prediction2'].split())
    REC.append(dt['prediction3'].split())
    REC.append(dt['prediction4'].split())
    REC.append(dt['prediction5'].split())

    #Create a dictionary of items recommended.
    #Assign a weight according the order of appearance and multiply by global weights
    res = {}
    for M in range(len(REC)):
        for n, v in enumerate(REC[M]):
            if v in res:
                res[v] += (W[M]/(n+1))
            else:
                res[v] = (W[M]/(n+1))

    # Sort dictionary by item weights
    res = list(dict(sorted(res.items(), key=lambda item: -item[1])).keys())

    # Return the top 12 items only
    return ' '.join(res[:12])

# weights assigned arbitrarily
sub1['prediction'] = sub1.apply(cust_blend, W = [1.05,0.78,0.86,0.88,0.85], axis=1)
sub1.head()

,customer_id,prediction1,prediction2,prediction3,prediction4,prediction5,prediction
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,0568601043 0924243001 0909370001 0865799006 09...,0568601043 0751471001 0909370001 0918522001 09...,0568601043 0568601006 0448509014 0573085028 07...,0568601043 0568601006 0745232001 0751471001 04...,0568601043 0568601006 0745232001 0751471001 04...,0568601043 0568601006 0751471001 0448509014 09...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,0924243001 0924243002 0918522001 0923758001 08...,0924243001 0924243002 0923758001 0918522001 09...,0826211002 0800436010 0739590027 0448509014 05...,0826211002 0739590027 0811835004 0764280001 07...,0826211002 0739590027 0811835004 0764280001 07...,0826211002 0924243001 0739590027 0924243002 08...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0794321007 0924243001 0762846027 0918522001 09...,0794321007 0924243001 0923758001 0924243002 09...,0794321007 0852643001 0852643003 0858883002 07...,0794321007 0858883002 0852643003 0727808007 08...,0794321007 0852643001 0852643003 0727808007 08...,0794321007 0852643001 0924243001 0852643003 08...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,0924243001 0924243002 0918522001 0923758001 08...,0924243001 0924243002 0923758001 0918522001 09...,0448509014 0573085028 0751471001 0706016001 06...,0751471001 0579541001 0573085028 0673677002 06...,0751471001 0678942001 0673677002 0579541001 05...,0751471001 0924243001 0448509014 0924243002 05...
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,0924243001 0924243002 0918522001 0923758001 08...,0924243001 0924243002 0923758001 0918522001 09...,0730683050 0791587015 0896152002 0818320001 09...,0730683050 0791587015 0896152002 0927530004 08...,0730683050 0791587015 0896152002 0927530004 05...,0730683050 0924243001 0791587015 0924243002 08...


In the above code, the weightages have been arbitrarily tweaked and submitted on Kaggle, in order to know if it brings improvement or not. 

# Make a submission

In [8]:
del sub1['prediction1']
del sub1['prediction2']
del sub1['prediction3']
del sub1['prediction4']
del sub1['prediction5']

gc.collect()


sub1.to_csv('../datasets/ensemble9.csv', index=False)

My 9th attempt at ensembling gave the highest MAP@12 at 0.0237

## Conclusion from ensembling

Further iterations of ensembles did not improve the MAP from 0.0237. As such, they will not be included in the notebook. Strategies like multiblend for ensemble was also employed, but did not improve the score.

It was interesting to note that the improvement in MAP@12 was 4.4% from the best performing model of 0.0227. I also noted that the more diverse the models, the better the score improves. 

## Future works

The competition has ended on 10 May 2022, and the top scoring methodologies are starting to be posted on the public boards.

It was noted that the top strategies employed multiple candidate generation method, some even up to 20, before employing a ranking model. Ensemble was also a crucial step that was employed by them. [*link*](https://www.kaggle.com/competitions/h-and-m-personalized-fashion-recommendations/discussion/324103)

For future works and learning, I would want to try to further improve the score by using these top ranking techniques. I noted that recommender competitions are pretty popular, and with this experience, I hope to score better in future in the leaderboards.